### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import pandas as pd
import numpy as np


# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#check to see if the data set is complete across columns
school_data_complete.count()

# lots of different checks to see if there are any duplicate entries in the dataset
used_features = ["student_name","school_name"]
school_data_complete["name_school_duplicated"] = school_data_complete.duplicated(used_features)
school_data_complete['name_school_duplicated'].sum()
# found that there are 880 entries in the data set that share student names and school entry in some fashion with the above code
school_data_complete.head()
# Create a dataframe of just the duplicate entries to examine
duplicate_df = school_data_complete[school_data_complete['name_school_duplicated'] == True]
duplicate_df

# Checks to see if each of the columns has equal amounts of data populated within it.  In this case, each column has 39170 entries
school_data_complete.count()

#Sorted duplicate data fram in ascending order based on student name.
#Based on a random selection of data, it looks like each entry that shares a name represents a unique student that is 
#in a different grade, different school, and has achieved different test scores
#Based on this I am going to assume there are no duplicate entries in this dataset.  
sorted_duplicate_df= duplicate_df.sort_values(['student_name','school_name'], ascending=True)
sorted_duplicate_df.iloc[200:225]

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,name_school_duplicated
28275,28275,David Walker,M,12th,Rodriguez High School,95,75,11,District,3999,2547363,True
4487,4487,David Watson,M,9th,Figueroa High School,99,83,1,District,2949,1884411,True
4031,4031,David White,M,12th,Figueroa High School,72,72,1,District,2949,1884411,True
21397,21397,David Wilson,M,11th,Bailey High School,89,93,7,District,4976,3124928,True
10304,10304,David Wilson,M,10th,Hernandez High School,64,75,3,District,4635,3022020,True
21790,21790,Dawn Hill,F,11th,Bailey High School,91,94,7,District,4976,3124928,True
1547,1547,Dawn Martinez,F,11th,Huang High School,67,66,0,District,2917,1910635,True
32445,32445,Deborah Johnson,F,9th,Johnson High School,86,81,12,District,4761,3094650,True
32693,32693,Deborah Johnson,F,9th,Johnson High School,79,85,12,District,4761,3094650,True
1872,1872,Dennis Brown,M,11th,Huang High School,92,76,0,District,2917,1910635,True


In [2]:
# use the unique pandas function on the "school_name" column to find out the unique entries in this column
unique_schools = school_data_complete["school_name"].unique()
print(len(unique_schools))
print(unique_schools)
# use the unique function on the size column to find out the unique entries in this column
unique_sizes = school_data_complete['size'].unique()
print(unique_sizes)

15
['Huang High School' 'Figueroa High School' 'Shelton High School'
 'Hernandez High School' 'Griffin High School' 'Wilson High School'
 'Cabrera High School' 'Bailey High School' 'Holden High School'
 'Pena High School' 'Wright High School' 'Rodriguez High School'
 'Johnson High School' 'Ford High School' 'Thomas High School']
[2917 2949 1761 4635 1468 2283 1858 4976  427  962 1800 3999 4761 2739
 1635]


In [3]:
#make a dataframe for school size indexed to school_name
schools_df = pd.DataFrame(unique_schools)
schools_df.rename(columns={0:'school_name'},inplace = True)
sizes_df = pd.DataFrame(unique_sizes)
sizes_df.rename(columns={0:'size'}, inplace = True)
school_sizes = pd.concat([schools_df, sizes_df], axis = 1)
school_sizes_df = school_sizes.set_index(school_sizes['school_name'])
school_sizes_df.drop('school_name', axis =1, inplace = True)
school_sizes_df

,size
school_name,
Huang High School,2917
Figueroa High School,2949
Shelton High School,1761
Hernandez High School,4635
Griffin High School,1468
Wilson High School,2283
Cabrera High School,1858
Bailey High School,4976
Holden High School,427


In [4]:
# use the value counts for the "student_name" category to find out how many unique names are in the column
unique_student_names = school_data_complete["student_name"].value_counts()
print(len(unique_student_names))
number_of_students = school_data_complete["student_name"].count()
print("{:,.0f}".format(number_of_students))


32715
39,170


In [5]:
#Budgets are tracked at the school level and each is a unique entry

unique_budget_lines = school_data_complete['budget'].unique()
unique_budget_lines.sum()
print("${:,.2f}".format(unique_budget_lines.sum()))
unique_budget_lines


$24,649,428.00


array([1910635, 1884411, 1056600, 3022020,  917500, 1319574, 1081356,
       3124928,  248087,  585858, 1049400, 2547363, 3094650, 1763916,
       1043130], dtype=int64)

In [6]:
#calculate the average math score using the mean function in Pandas
average_math_score = school_data_complete['math_score'].mean()
print(average_math_score)
print("{:.6f}".format(average_math_score))


78.98537145774827
78.985371


In [7]:
#calculate the average reading score using the mean function in Pandas
average_reading_score = school_data_complete['reading_score'].mean()
print(average_reading_score)
print("{:.6f}".format(average_reading_score))

81.87784018381414
81.877840


In [8]:
#finds the number of students who passed the math test if the passing grade is 70
math_students_passed_df = school_data_complete[school_data_complete['math_score'] >= 70 ]
#passing % equal number of passing students/total students
passing_math_students = int(len(math_students_passed_df))/int(len(school_data_complete))
print(passing_math_students*100)
print("{:.6f}".format(passing_math_students*100))
passing_math_students=passing_math_students*100


74.9808526933878
74.980853


In [9]:
#finds the number of students who passed the reading test if the passing grade is 70
reading_students_passed_df = school_data_complete[school_data_complete['reading_score'] >= 70 ]
#passing % equal number of passing students/total students
passing_reading_students = int(len(reading_students_passed_df))/int(len(school_data_complete))
print(passing_reading_students*100)
print("{:.6f}".format(passing_reading_students*100))
passing_reading_students = passing_reading_students * 100

85.80546336482001
85.805463


In [10]:
#finds the number of students who passed both tests by merging data frames with common student ids
passing_students_df = pd.merge(math_students_passed_df, reading_students_passed_df, on=["Student ID"])
#find passing rate by calculating length of new data frame and dividing by the length of the orginal dataframe
passing_rate = int(len(passing_students_df))/int(len(school_data_complete))
print(passing_rate*100)
print("{:.6f}".format(passing_rate*100))
passing_rate = passing_rate * 100
passing_students_df.head()


65.17232575950983
65.172326


,Student ID,student_name_x,gender_x,grade_x,school_name_x,reading_score_x,math_score_x,School ID_x,type_x,size_x,...,gender_y,grade_y,school_name_y,reading_score_y,math_score_y,School ID_y,type_y,size_y,budget_y,name_school_duplicated_y
0,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,...,F,9th,Huang High School,97,84,0,District,2917,1910635,False
1,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,...,M,9th,Huang High School,94,94,0,District,2917,1910635,False
2,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,...,F,11th,Huang High School,82,80,0,District,2917,1910635,False
3,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,...,M,10th,Huang High School,95,87,0,District,2917,1910635,False
4,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,...,M,10th,Huang High School,96,84,0,District,2917,1910635,False


In [ ]:
total_table_df = pd.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [11]:
summary_df = pd.DataFrame({
               "Total Schools":[len(unique_schools)],
               "Total Students":[number_of_students],
               "Total Budget":[unique_budget_lines.sum()],
               "Average Math Score":[average_math_score],
               "Average Reading Score":[average_reading_score],
               '% Passing Math':[passing_math_students],
               '% Passing Reading':[passing_reading_students],
               '% Overall Passing':[passing_rate]
               })
summary_df['Total Students'] = summary_df['Total Students'].astype(float).map("{:,.0f}".format)
summary_df['Total Budget'] = summary_df['Total Budget'].astype(float).map("${:,.2f}".format)
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [12]:
school_type = []
for j in unique_schools:
    flag = 0
    for i in school_data_complete['Student ID']:
        if school_data_complete['school_name'][i] == j and flag == 0:
           typeholder = school_data_complete['type'][i]
           school_type.append(typeholder)
           flag = 1
school_type
type_school = pd.DataFrame(school_type, columns = ['type'])
unique_schools = pd.DataFrame(unique_schools, columns = ['school_name'])

#type_school.merge(unique_schools, on = 'index')
#type_school.head()
#unique_schools.head()
#merged_type_school = type_school.astype(str) + unique_schools.astype(str)
school_type
school_type
unique_schools
type_school
#type_school = type_school.set_index(unique_schools['school_name'])
#merged_type_school.head()
merged_type_school = pd.merge(type_school,unique_schools, right_index = True, left_index = True)
merged_type_school = merged_type_school.set_index(unique_schools['school_name'])

merged_type_school.head()
#type_school



,type,school_name
school_name,,
Huang High School,District,Huang High School
Figueroa High School,District,Figueroa High School
Shelton High School,Charter,Shelton High School
Hernandez High School,District,Hernandez High School
Griffin High School,Charter,Griffin High School


In [13]:
#assign a new column to master dataframe to calculate if a student passed math
math_pf = []
for i in school_data_complete['Student ID']:  
    if school_data_complete['math_score'][i] >= 70:
        math_pf.append(1)
    else:
        math_pf.append(0)
school_data_complete['Passed Math'] = math_pf
school_data_complete.head()

#assign a new column to master dataframe to calculate if a student passed math
reading_pf = []
for i in school_data_complete['Student ID']:  
    if school_data_complete['reading_score'][i] >= 70:
        reading_pf.append(1)
    else:
       reading_pf.append(0)
school_data_complete['Passed Reading'] = reading_pf
school_data_complete.head()

#Group by school statement
group_by_school_df = school_data_complete.groupby(["school_name"])
group_by_school_df.count()
school_summary = group_by_school_df
school_summary = school_summary[['Student ID','reading_score','math_score','type','budget','Passed Math','Passed Reading']]
 
#create df for the student counts
student_counts = school_data_complete.groupby(['school_name']).count()['Student ID']
student_counts

#Calculate budget per school
school_budgets = school_data_complete.groupby(['school_name']).sum()['budget']/student_counts
school_budgets

#Calculate budget per student
budget_per_student = school_budgets / student_counts
budget_per_student

#Calculate Average Math Score per student by school
school_math_scores = school_data_complete.groupby(['school_name']).sum()['math_score']
average_mathscore_school = school_math_scores / student_counts
average_mathscore_school

#Calculate Average Reading Score per student by school
school_reading_scores = school_data_complete.groupby(['school_name']).sum()['reading_score']
average_readingscore_school = school_reading_scores / student_counts
average_readingscore_school
#school_summary.count()

#Calculate the % of students who are passing math at each school

#assign a new column to master dataframe to calculate if a student passed math
math_pf = []
for i in school_data_complete['Student ID']:  
    if school_data_complete['math_score'][i] >= 70:
        math_pf.append(1)
    else:
        math_pf.append(0)
school_data_complete['Passed Math'] = math_pf
school_data_complete.head()

#assign a new column to master dataframe to calculate if a student passed math
reading_pf = []
for i in school_data_complete['Student ID']:  
    if school_data_complete['reading_score'][i] >= 70:
        reading_pf.append(1)
    else:
       reading_pf.append(0)
school_data_complete['Passed Reading'] = reading_pf
school_data_complete.head()

#Calculate the % of students who are passing math at each school
school_passed_math_scores =school_data_complete.groupby(['school_name']).sum()['Passed Math'] 
school_passing_math_rate = school_passed_math_scores / student_counts
school_passing_math_rate

#Calculate the % of students who are passing reading at each school
school_passed_reading_scores =school_data_complete.groupby(['school_name']).sum()['Passed Reading'] 
school_passing_reading_rate = school_passed_reading_scores / student_counts
school_passing_reading_rate

#Find passing students by utilizing earlier merged dataframe of only passing students
overall_passing_rates = passing_students_df.groupby('school_name_x').count()['Student ID'] 
school_passing_rates = overall_passing_rates / student_counts
#school_passing_rates.rename(columns={'school_name_x':'school_name',' ':'% Overall Passing'})
school_passing_rates



school_name_x
Bailey High School       0.546423
Cabrera High School      0.913348
Figueroa High School     0.532045
Ford High School         0.542899
Griffin High School      0.905995
Hernandez High School    0.535275
Holden High School       0.892272
Huang High School        0.535139
Johnson High School      0.535392
Pena High School         0.905405
Rodriguez High School    0.529882
Shelton High School      0.898921
Thomas High School       0.909480
Wilson High School       0.905826
Wright High School       0.903333
Name: Student ID, dtype: float64

In [17]:
#Construct table that summarizes metrics

#convert student counts into a data frame and rename the column
student_counts = pd.DataFrame(student_counts)
student_counts.rename(columns={"Student ID":"Total Students"}, inplace = True)

#merge the type of school data frame with the student count dataframe and then drop the redundant school_name column
school_summary_table_df = pd.merge(merged_type_school,student_counts,left_index=True, right_index=True)
school_summary_table_df
school_summary_table_df.drop('school_name', axis =1, inplace = True)
school_summary_table_df

#convert total budgets into a data frame and rename the column
school_budgets = pd.DataFrame(school_budgets)
school_budgets.rename(columns={0:'Total School Budget'}, inplace = True)

#merge total budget data frame into school summary 
school_summary_table_df = pd.merge(school_summary_table_df, school_budgets, left_index = True, right_index = True)
school_summary_table_df

#convert budget per student into a dataframe and rename the column
budget_per_student = pd.DataFrame(budget_per_student)
budget_per_student.rename(columns={0:'Per Student Budget'}, inplace = True)
budget_per_student

#merge budget per student into school summary
school_summary_table_df = pd.merge(school_summary_table_df, budget_per_student, left_index = True, right_index = True)
school_summary_table_df

#convert average math score to a dataframe and rename the column
average_mathscore_school = pd.DataFrame(average_mathscore_school)
average_mathscore_school.rename(columns={0:'Average Math Score'}, inplace = True)

#merge average math score into school summary
school_summary_table_df = pd.merge(school_summary_table_df, average_mathscore_school, left_index = True, right_index = True)
school_summary_table_df

#convert average reading score to a dataframe and rename the column
average_readingscore_school = pd.DataFrame(average_readingscore_school)
average_readingscore_school.rename(columns={0:'Average Reading Score'}, inplace = True)

#merge average reading score into school summary
school_summary_table_df = pd.merge(school_summary_table_df, average_readingscore_school, left_index = True, right_index = True)
school_summary_table_df

#convert % passing math to a dataframe and rename the column
school_passing_math_rate = pd.DataFrame(school_passing_math_rate)
school_passing_math_rate.rename(columns={0:'% Passing Math'}, inplace = True)

#merge % passing math score into school summary
school_summary_table_df = pd.merge(school_summary_table_df, school_passing_math_rate, left_index = True, right_index = True)
school_summary_table_df

#convert % passing reading to a dataframe and rename the column
school_passing_reading_rate = pd.DataFrame(school_passing_reading_rate)
school_passing_reading_rate.rename(columns={0:'% Passing Reading'}, inplace = True)

#merge % passing reading score into school summary
school_summary_table_df = pd.merge(school_summary_table_df, school_passing_reading_rate, left_index = True, right_index = True)
school_summary_table_df

#convert % overall passing to a dataframe and rename the column
school_passing_rates = pd.DataFrame(school_passing_rates)
school_passing_rates.rename(columns={"Student ID":'% Overall Passing'}, inplace = True)

#merge % overall passing score into school summary
school_summary_table_df = pd.merge(school_summary_table_df, school_passing_rates, left_index = True, right_index = True)
school_summary_table_df

#sort table by index in ascending order
school_summary_table_df.sort_index()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Shelton High School,Charter,1761,1056600.0,600.0,83.359455,83.725724,0.938671,0.958546,0.898921
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [ ]:
* Sort and display the top five performing schools by % overall passing.


In [19]:
#Create a sort on % overall passing and hold value in place since I am not creating a new data frame
school_summary_table_df
school_summary_table_df.sort_values(by='% Overall Passing', ascending = 0, inplace = True)
school_summary_table_df.head()


,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [20]:
#Create a sort on % overall passing and hold value in place since I am not creating a new data frame

school_summary_table_df.sort_values(by='% Overall Passing', ascending = 1, inplace = True)
school_summary_table_df.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [21]:
school_summary_table_df
# make a 9th grade dataframe
school_type = []
for j in unique_schools:
    flag = 0
    for i in school_data_complete['Student ID']:
        if school_data_complete['grade'][i] == j and flag == 0:
           typeholder = school_data_complete['type'][i]
           school_type.append(typeholder)
           flag = 1
school_type

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.812224,0.535392
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,0.683096,0.792990,0.542899
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Shelton High School,Charter,1761,1056600.0,600.0,83.359455,83.725724,0.938671,0.958546,0.898921
Wright High School,Charter,1800,1049400.0,583.0,83.682222,83.955000,0.933333,0.966111,0.903333


In [35]:
math_pivot = pd.pivot_table(school_data_complete,index=['school_name'],columns=['grade'],values=['math_score'], aggfunc = np.mean)
math_pivot

math_score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [33]:
reading_pivot = pd.pivot_table(school_data_complete,index=['school_name'],columns=['grade'],values=['reading_score'], aggfunc = np.mean)
reading_pivot 


reading_score                                 
grade                          10th       11th       12th        9th
school_name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [495]:
#find max spending per student value for bin use
#school_summary_table_df['Per Student Budget'].max()
#max was found to be 655 which is within the max bin value of 680

#create spending bins
spending_bins = [0,585,630,645,680]

#create names for bins
spending_bin_names = ['<$585','$585-630','$630-645','$645-680']

#Create a column that has the bin values that are wanted
school_summary_table_df['Scores by School Spending'] = pd.cut(school_summary_table_df['Per Student Budget'], spending_bins, labels = spending_bin_names, include_lowest = True)
#school_summary_table_df

#make a groupby the Scores by School Spending column
labels = ['Total Students','Total School Budget','Per Student Budget']
scores_by_school_spending = school_summary_table_df.groupby('Scores by School Spending')

#show just the columns we are interested in 
scores_by_school_spending['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading',
'% Overall Passing'].mean()


ipykernel_launcher:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Scores by School Spending,,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.903695
$585-630,81.899826,83.155286,0.871335,0.927182,0.814186
$630-645,78.518855,81.624473,0.734842,0.843918,0.628577
$645-680,76.997210,81.027843,0.661648,0.811340,0.535269


## Scores by School Size

* Perform the same operations as above, based on school size.

In [506]:
Size_summary_df = school_summary_table_df
#Activate this next line when running the code from scratch
# #Size_summary_df.drop('Scores by School Spending', axis = 1, inplace = True)


Size_summary_df = pd.concat([Size_summary_df,school_sizes_df], axis =1)
Size_summary_df

#create size bins
size_bins = [0,1000,2000,5000]

#create names for bins
size_bin_names = ['Small (<1000)','Medium (1000-2000)','Large(2000-5000)']

#Create a column that has the bin values that are wanted
Size_summary_df['School Size'] = pd.cut(Size_summary_df['size'], size_bins, labels = size_bin_names, include_lowest = True)

#make a groupby the Scores by School Spending column
Size_summary_df = Size_summary_df.groupby('School Size')

#show just the columns we are interested in 
Size_summary_df['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading',
'% Overall Passing'].mean()

ipykernel_launcher:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large(2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [508]:
Type_summary_df = school_summary_table_df
#Activate this next line when running the code from scratch
#Type_summary_df.drop('Scores by School Spending', axis = 1, inplace = True)

Type_summary_df = Type_summary_df.groupby('type')

#show just the columns we are interested in 
Type_summary_df['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading',
'% Overall Passing'].mean()

ipykernel_launcher:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
